In [1]:
import numpy as np
import glob
import pypianoroll as ppr
import time
import music21
import os
import torch
import torch.utils.data

from torch import nn, optim
from torch.nn import functional as F
from utils.utilsPreprocessing import *
#np.set_printoptions(threshold=np.inf)
#torch.set_printoptions(threshold=50000)

In [2]:
##################################
#HYPERPARAMS
##################################
epochs = 1
learning_rate = 1e-4
batch_size= 98
log_interval = 1  #Log/show loss per batch

# Load MIDI files from npz

In [3]:
data = np.load('/Volumes/EXT/DATASETS/WikifoniaPartlyNoTranspose.npz')

midiDatasetTrain = data['train']
midiDatasetTest = data['test']

data.close()

"""
print("Training set: ({}, {}, {}, {})".format(midiDatasetTrain.size()[0],
                                                midiDatasetTrain.size()[1],
                                                midiDatasetTrain.size()[2],
                                                midiDatasetTrain.size()[3]))
print("Test set: ({}, {}, {}, {})".format(midiDatasetTest.size()[0],
                                                midiDatasetTest.size()[1],
                                                midiDatasetTest.size()[2],
                                                midiDatasetTest.size()[3]))
"""

print("Training set: {}".format(midiDatasetTrain.shape))
print("Test set: {}".format(midiDatasetTest.shape))

Training set: (4056, 1, 96, 60)
Test set: (1034, 1, 96, 60)


In [4]:
fullPitch = 128
_, _, length, reducedPitch = midiDatasetTrain.shape

# CDVAE

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
midiDatasetTrain = torch.from_numpy(midiDatasetTrain)
trainLoader = torch.utils.data.DataLoader(midiDatasetTrain, batch_size=batch_size, shuffle=False, drop_last=True)

midiDatasetTest = torch.from_numpy(midiDatasetTest)
testLoader = torch.utils.data.DataLoader(midiDatasetTest, batch_size=batch_size, shuffle=False, drop_last=True)

In [9]:
class CDVAE(nn.Module):
    def __init__(self, batch_size=7, tie_weights=True):
        super(CDVAE, self).__init__()
        
        self.batch_size = batch_size
        
        ###ENCODER###
        self.encode1 = nn.Sequential(
            nn.Conv2d(1,100,(16,5),stride=(16,5),padding=0),
            nn.BatchNorm2d(100),
            nn.ELU(),
            nn.Conv2d(100,200,(2,1),stride=(2,1),padding=0),
            nn.BatchNorm2d(200),
            nn.ELU(),
            nn.Conv2d(200,400,(2,2),stride=(1,2),padding=0),
            nn.BatchNorm2d(400),
            nn.ELU(),
            nn.Conv2d(400,800,(2,2),stride=(2,2),padding=0),
            nn.BatchNorm2d(800),
            nn.ELU()
        )
            
        self.encode2 = nn.Sequential(
            nn.Linear(2400,800),
            nn.BatchNorm1d(800),
            nn.ELU(),
            nn.Linear(800,400),
            nn.BatchNorm1d(400),
            nn.ELU(),
            nn.Linear(400,100),
            nn.BatchNorm1d(100),
            nn.ELU()
        )

        ###DECODER###
        self.decode1 = nn.Sequential(
            nn.Linear(100,400),
            nn.BatchNorm1d(400),
            nn.ELU(),
            nn.Linear(400,800),
            nn.BatchNorm1d(800),
            nn.ELU(),
            nn.Linear(800,2400),
            nn.BatchNorm1d(2400),
            nn.ELU()
        )
        self.decode2 = nn.Sequential(
            nn.ConvTranspose2d(800,400,(2,2),stride=(2,2),padding=0),
            nn.BatchNorm2d(400),
            nn.ELU(),
            nn.ConvTranspose2d(400,200,(2,2),stride=(1,2),padding=0),
            nn.BatchNorm2d(200),
            nn.ELU(),
            nn.ConvTranspose2d(200,100,(2,1),stride=(2,1),padding=0),
            nn.BatchNorm2d(100),
            nn.ELU(),
            nn.ConvTranspose2d(100,1,(16,5),stride=(16,5),padding=0),
            nn.BatchNorm2d(1),
            nn.ELU()
        )
        
        ###LSTM###
        self.lstm1 = nn.LSTM(input_size=100, hidden_size=400, num_layers=2)
        self.lstm2 = nn.LSTM(input_size=400, hidden_size=100, num_layers=1)
    
    def encoder(self, x):
        #print("ENOCDER")
        hEnc = self.encode1(x)
        #print(hEnc.size())
        hEnc = torch.squeeze(hEnc,3).view(-1,800*3)
        #print(hEnc.size())
        hEnc = self.encode2(hEnc)
        return hEnc

    def decoder(self, z):
        #print("DECODER")
        hDec = self.decode1(z)
        hDec = hDec.view(hDec.size()[0],800,-1).unsqueeze(2)
        hDec = self.decode2(hDec)
        return hDec


    def forward(self, x):
        embed = self.encoder(x)
        
        ####MOVE TO HIDDEN_INIT
        h_t = torch.zeros(2,int(embed.size()[0]/7),400).to(device)
        c_t = torch.zeros(2,int(embed.size()[0]/7),400).to(device)
        h2_t = torch.zeros(1,int(embed.size()[0]/7),100).to(device)
        c2_t = torch.zeros(1,int(embed.size()[0]/7),100).to(device)        
        ###HIDDEN INIT END
        
        #IF FOR TESTING UNKNOWN SEQUENCES
        if(embed.size()[0]>7):
            embedTemp = torch.chunk(embed, int(self.batch_size/7),dim=0)
            #print(len(embedTemp))
            embed7s = embedTemp[0].unsqueeze(1)
            for emb in embedTemp[1:]:
                embed7s = torch.cat((embed7s, emb.unsqueeze(1)),dim=1)
        else:
            embed7s = embed.unsqueeze(1)
            
        lstmOut, (h_t, c_t) = self.lstm1(embed7s,(h_t, c_t))
        lstmOut, (h2_t, c2_t) = self.lstm2(lstmOut,(h2_t, c2_t))
        
        return embed, lstmOut, self.decoder(lstmOut[-1,:,:])
    
model = CDVAE(batch_size=batch_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


def loss_function(embed, lstmOut, embedNext):
    
    cosLSTM = nn.CosineSimilarity(dim=0, eps=1e-8)
    
    cosSimLSTM = 0
    #BATCHSIZE 7
    if(embed.size()[0]==7):
        #print("batchsize = 7 ?")
        cosSimLSTM = cosLSTM(lstmOut.squeeze(1)[-1], embedNext[0])
     
    #BATCHSIZE > 7
    else:
        j=0
        for i in range(7,embed.size()[0],7):
            #print(i)
            cosSimLSTM += cosLSTM(lstmOut[-1,j,:],embed[i])
            j+=1
        cosSimLSTM += cosLSTM(lstmOut[-1,j,:],embedNext[0])
    
    return -cosSimLSTM/lstmOut.size()[1]
        

def train(epoch):
    model.train()
    trainLoss = 0

    for batch_idx, data in enumerate(trainLoader):
        #print(batch_idx)
        data = data.float().to(device)
        optimizer.zero_grad()
        embedding, lstmOut, reconPrediction = model(data)
        nextBatch = next(iter(trainLoader)).float().to(device)
        with torch.no_grad():
            embeddingNext, _, _ = model(nextBatch)
        #print(nextBatch.size())
        loss = loss_function(embedding, lstmOut, embeddingNext)
        loss.backward()
        trainLoss += loss.item()
        optimizer.step()
        if(batch_idx % log_interval == 0):
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tAccuracy: {:.6f}'.format(
                epoch, batch_idx * len(data), len(trainLoader.dataset),
                100. * batch_idx / len(trainLoader),
                -loss.item() / len(data)))
        if(batch_idx==10):
            break
    print('====> Epoch: {} Average accuracy: {:.4f}'.format(
          epoch, -trainLoss / len(trainLoader.dataset)/lstmOut.size()[0]))

def test(epoch):
    model.eval()
    testLoss = 0
    with torch.no_grad():
        for i, data in enumerate(testLoader):
            data = data.float().to(device)
            embedding, lstmOut, reconPrediction = model(data)
            nextBatch = next(iter(testLoader)).float().to(device)
            embeddingNext, _, _= model(nextBatch)
            testLoss += loss_function(embedding, lstmOut, embeddingNext).item()
            
            if(i==10):
                break
    testLoss /= (len(testLoader.dataset)/lstmOut.size()[0])

    print('====> Test set accuracy: {:.4f}'.format(-testLoss))

In [10]:
"""
#LOAD MODEL
pathToModel = 'model/YamahaPianoComp2002_20Epochs_LSTM_noTW.model'

try:
    #LOAD TRAINED MODEL INTO GPU
    if(torch.cuda.is_available()):
        model = torch.load(pathToModel)
        
    #LOAD MODEL TRAINED ON GPU INTO CPU
    else:
        model = torch.load(pathToModel, map_location=lambda storage, loc: storage)
    print("\n--------model restored--------\n")
except:
    print("\n--------no saved model found--------\n")
"""
print('')

In [11]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)

Train Epoch: 1 [0/4056 (0%)]	Accuracy: 0.000031
Train Epoch: 1 [98/4056 (2%)]	Accuracy: 0.000143
Train Epoch: 1 [196/4056 (5%)]	Accuracy: -0.000382
Train Epoch: 1 [294/4056 (7%)]	Accuracy: 0.000826
Train Epoch: 1 [392/4056 (10%)]	Accuracy: 0.000192
Train Epoch: 1 [490/4056 (12%)]	Accuracy: 0.000893
Train Epoch: 1 [588/4056 (15%)]	Accuracy: 0.000171
Train Epoch: 1 [686/4056 (17%)]	Accuracy: 0.000615
Train Epoch: 1 [784/4056 (20%)]	Accuracy: 0.000490
Train Epoch: 1 [882/4056 (22%)]	Accuracy: 0.000524
Train Epoch: 1 [980/4056 (24%)]	Accuracy: 0.000340
====> Epoch: 1 Average accuracy: 0.0000
====> Test set accuracy: 0.0010


In [ ]:
#torch.save(model,'/media/EXTHD/niciData/models/Yamaha10Epochs_LSTM.model')

In [ ]:
#np.set_printoptions(precision=2, suppress=True, threshold=np.inf)


In [ ]:
###PLAY WHOLE SONG IN BARS
with torch.no_grad():
    
    sampleNp1 = getSlicedPianorollMatrixNp("/Volumes/EXT/DATASETS/WikifoniaServer/test/Claude-Debussy---Reverie.mid")
    sampleNp1 = deleteZeroMatrices(sampleNp1)
    sample = np.expand_dims(sampleNp1[0,:,36:-32],axis=0)
    print(sample.shape)
    for i, sampleNp in enumerate(sampleNp1[1:7]):
        #print(sampleNp.shape)
        print(i)
        if(np.any(sampleNp)):
            sampleNp = sampleNp[:,36:-32]
            sampleNp = np.expand_dims(sampleNp,axis=0)
            sample = np.concatenate((sample,sampleNp),axis=0)
            print(sample.shape)
    samplePlay = sample[0,:,:]
    for s in sample[1:]:
        samplePlay = np.concatenate((samplePlay,s),axis=0)
    samplePlay = addCuttedOctaves(samplePlay)
    print(samplePlay.shape)
    sample = torch.from_numpy(sample).float().to(device)
    sample = torch.unsqueeze(sample,1)
    print(sample.size())
    _,_, pred = model(sample)
    #reconstruction = recon.squeeze(0).squeeze(0).cpu().numpy()
    prediction = pred.squeeze(0).squeeze(0).cpu().numpy()

    #print(sampleNp[:,:])
    #print(prediction[:,:])
    #print(np.sum(sampleNp.numpy(), axis=1))

    #NORMALIZE PREDICTIONS
    #reconstruction /= np.abs(np.max(reconstruction))
    prediction /= np.abs(np.max(prediction))
    #print(prediction)

    #CHECK MIDI ACTIVATIONS IN PREDICTION TO INCLUDE RESTS
    #reconstruction[reconstruction < 0.3] = 0
    prediction[prediction < 0.8] = 0



    ###MONOPHONIC OUTPUT MATRIX POLOYPHONIC POSSIBLE WITH ACTIVATION THRESHOLD###
    #score = music21.converter.parse('WikifoniaServer/samples/The-Doors---Don\'t-you-love-her-Madly?.mid')
    #score.show()

    samplePlay = debinarizeMidi(samplePlay, prediction=False)
    samplePlay = addCuttedOctaves(samplePlay)
    #reconstruction = debinarizeMidi(reconstruction, prediction=True)
    #reconstruction = addCuttedOctaves(reconstruction)
    prediction = debinarizeMidi(prediction, prediction=True)
    prediction = addCuttedOctaves(prediction)

    #print(np.argmax(samplePlay, axis=1))
    #print('')
    #print(np.argmax(prediction, axis=1))
    print("INPUT")
    print(samplePlay.shape)
    pianorollMatrixToTempMidi(samplePlay)
    tempMidi(show=True,play=True)
    #print("RECONSTRUCTION")
    #pianorollMatrixToTempMidi(reconstruction)        
    #tempMidi(show=True,play=True)
    print("PREDICTION")
    pianorollMatrixToTempMidi(prediction, prediction=True)        
    tempMidi(show=True,play=True)
    print("\n\n")
            

print('')